**URMP Dataset** - Multi-track classical music performance dataset

---



Importamos los datos de la base de datos de Dryad Data:

In [ ]:
#!wget https://datadryad.org/stash/downloads/file_stream/100292

--2020-11-18 18:25:16--  https://datadryad.org/stash/downloads/file_stream/100292
Resolving datadryad.org (datadryad.org)... 52.42.80.64, 44.236.162.222, 44.239.128.219
Connecting to datadryad.org (datadryad.org)|52.42.80.64|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://dryad-assetstore-merritt-west.s3.us-west-2.amazonaws.com/ark%3A/13030/m5cc634w%7C1%7Cproducer/Dataset.tar.gz?response-content-type=application%2Fx-gtar&X-Amz-Security-Token=IQoJb3JpZ2luX2VjENH%2F%2F%2F%2F%2F%2F%2F%2F%2F%2FwEaCXVzLXdlc3QtMiJGMEQCIGYk7KpCTwTbHLP1x2%2FprFxfLzIvqOv%2Fag2Y1WOJsaUZAiBYP%2FOyAt0jgUHwSQPGDgIYHS16z4kSMJ2GUVGgAKDwJiq0AwhKEAAaDDQ1MTgyNjkxNDE1NyIMfvmZ5Ww8dJlqDlu4KpEDs%2FJO3jdxzwUeR0YNR9%2F9%2Fj7uWITmwK7iX1pg1jy%2FHsELk%2Fhqfw72Rk8wqwzd5beW6ZqjXo9nBLJghvNIZ76gJlA259vhVmzboE08kebumziM%2FpUXC07%2FYIn9hRy9vPkimpW7vdIeed6f4OK9gt7VUd4XLUIIMycnaNxPmtK%2FNntgfm3ywgday8dmgXvOlsSLsyv1ZoEoH89mbh37430qNUYohc8Hgp67Zw6zUPWvb5bhZKB8J42S8%2FcwZRIovV3dM1U5PISm5wcueH7zP87A8zw

Descomprimimos:

In [ ]:
#!tar -zxf 100292

Movemos el dataset a nuestra unidad de Google Drive, para poder trabajar con ella fácilmente más tarde:

In [ ]:
#!mv -v Dataset/* drive/MyDrive/TFG/URMP-Dataset

Copiamos el dataset de nuestro entorno de Google Drive al entorno de ejecución de Google Colab (carpeta data):

In [ ]:
from google.colab import drive
drive.mount("drive")

Drive already mounted at drive; to attempt to forcibly remount, call drive.mount("drive", force_remount=True).


In [ ]:
!mkdir data
!cp -r drive/MyDrive/TFG/URMP-Dataset/* ./data

Prueba de **audio**:

In [ ]:
datafilename = "/content/drive/MyDrive/TFG/URMP-Dataset/01_Jupiter_vn_vc/AuSep_1_vn_01_Jupiter.wav"
from IPython.display import Audio
Audio(datafilename)

Output hidden; open in https://colab.research.google.com to view.

Prueba de **midi**:

In [ ]:
!apt install fluidsynth
!cp /usr/share/sounds/sf2/FluidR3_GM.sf2 ./font.sf2
!pip install midi2audio
from midi2audio import FluidSynth
from IPython.display import Audio

In [ ]:
FluidSynth("font.sf2").midi_to_audio('/content/drive/MyDrive/TFG/URMP-Dataset/01_Jupiter_vn_vc/Sco_01_Jupiter_vn_vc.mid', 'test.wav')
Audio("test.wav")

Output hidden; open in https://colab.research.google.com to view.

Prueba del **método Lucas-Kanade** para la detección del **optical flow** sobre un conjunto de características dispersas.

In [ ]:
import numpy as np
import cv2 as cv
from google.colab.patches import cv2_imshow
cap = cv.VideoCapture(cv.samples.findFile("/content/drive/MyDrive/TFG/URMP-Dataset/01_Jupiter_vn_vc/Vid_01_Jupiter_vn_vc.mp4"))
ret, frame1 = cap.read()
prvs = cv.cvtColor(frame1,cv.COLOR_BGR2GRAY)
hsv = np.zeros_like(frame1)
hsv[...,1] = 255
while(1):
    ret, frame2 = cap.read()
    next = cv.cvtColor(frame2,cv.COLOR_BGR2GRAY)
    flow = cv.calcOpticalFlowFarneback(prvs,next, None, 0.5, 3, 15, 3, 5, 1.2, 0)
    mag, ang = cv.cartToPolar(flow[...,0], flow[...,1])
    hsv[...,0] = ang*180/np.pi/2
    hsv[...,2] = cv.normalize(mag,None,0,255,cv.NORM_MINMAX)
    bgr = cv.cvtColor(hsv,cv.COLOR_HSV2BGR)
    cv2_imshow(bgr)
    k = cv.waitKey(30) & 0xff
    if k == 27:
        break
    elif k == ord('s'):
        cv.imwrite('opticalfb.png',frame2)
        cv.imwrite('opticalhsv.png',bgr)
    prvs = next

Output hidden; open in https://colab.research.google.com to view.

Prueba del **reconocimiento de la mano** mediante clasificadores en **Cascada de Haar**. 

In [ ]:
import cv2 
import numpy as np 
  
# cap = cv2.VideoCapture(0)
cap = cv.VideoCapture(cv.samples.findFile("/content/drive/MyDrive/TFG/URMP-Dataset/01_Jupiter_vn_vc/Vid_01_Jupiter_vn_vc.mp4"))
hand_cascade = cv2.CascadeClassifier('/content/drive/MyDrive/TFG/XML-CascadaDeHaar/palm.xml') 

count = 0

while(True): 
    ret, frame = cap.read() 
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) 
    hands = hand_cascade.detectMultiScale(gray, 1.5, 2) 
    # contour = hands 
    # contour = np.array(contour) 
  
    # if count==0: 
  
    #     if len(contour)==2: 
    #         cv2.putText(img=frame, text='Your engine started', 
    #                     org=(int(100 / 2 - 20), int(100 / 2)), 
    #                     fontFace=cv2.FONT_HERSHEY_DUPLEX,  
    #                     fontScale=1, color=(0, 255, 0)) 
                          
    #         for (x, y, w, h) in hands: 
    #             cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2) 
    #         count += 1
  
    # if count>0: 
  
    #     if len(contour)>=2: 
    #         cv2.putText(img=frame, text='You can take your car on long drive', 
    #                     org=(int(100 / 2 - 20), int(100 / 2)), 
    #                     fontFace=cv2.FONT_HERSHEY_DUPLEX,  
    #                     fontScale=1, color=(255, 0, 0)) 
                          
    #         for (x, y, w, h) in hands: 
    #             cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2) 
              
  
    #     elif len(contour)==1: 
    #         cv2.putText(img=frame, text='You can speed upto 80km/h', 
    #                     org=(int(100 / 2 - 20), int(100 / 2)), 
    #                     fontFace=cv2.FONT_HERSHEY_DUPLEX,  
    #                     fontScale=1, color=(0, 255, 0)) 
                          
    #         for (x, y, w, h) in hands: 
    #             cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2) 
  
    #     elif len(contour)==0: 
    #         cv2.putText(img=frame, text='Brake is applied slowly', 
    #                     org=(int(100 / 2 - 20), int(100 / 2)), 
    #                     fontFace=cv2.FONT_HERSHEY_DUPLEX,  
    #                     fontScale=1, color=(0, 0, 255)) 
  
    #     count += 1

    for (x, y, w, h) in hands: 
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2) 
  
    cv2_imshow(frame) 
    k = cv2.waitKey(30) & 0xff
    if k == 27: 
        break

Output hidden; open in https://colab.research.google.com to view.